# PLAT

## Packages

The demography package allows you to access data from directly from the [Human Mortality Database](https://www.mortality.org/). The StMoMo package allows one to fit a variety of mortality models.

In [4]:
library(StMoMo) # used to fit models
library(demography) # used to access data from the mortality database
library(TSA) # time series analysis
library(forecast) # time series forecast package
library(rjson)

## Data

We import the data for _England and Whales_. Since we're aiming to fit the Poisson and Negative Binomial versions of the Lee-Carter model, we'll be using the central mortality data.

In [10]:
cred <- fromJSON(file="../hmd_credentials.json")
username <- cred$username
password <- cred$password

country_code = "GBRTENW"

EWdata <- hmd.mx(country = country_code, username = username, 
                 password = password, label = country_code)

# central exposure data
EWMaleCenData <- StMoMoData(EWdata, series = "male")
EWFemaleCenData <- StMoMoData(EWdata, series = "female")

## Model

### Defining the Model

Ages, years and cohorts to be fitted

In [6]:
ages <- 19:103
years <- 1950:2018
cohorts <- (years[1] - ages[length(ages)]):(years[length(years)] - ages[1])
zero.cohorts <- cohorts[cohorts < 1855]
wxt <- genWeightMat(ages = ages, years = years, zeroCohorts = zero.cohorts)

In [ ]:
f2 <- function(x, ages) mean(ages) - x
f3 <- function(x, ages) pmax(mean(ages)-x,0)

constPlat <- function(ax, bx, kt, b0x, gc, wxt, ages){
  nYears <- dim(wxt)[2]
  x <- ages
  t <- 1:nYears
  c <- (1 - tail(ages, 1)):(nYears - ages[1])
  xbar <- mean(x)
  #\sum g(c)=0, \sum cg(c)=0, \sum c^2g(c)=0
  phiReg <- lm(gc ~ 1 + c + I(c^2), na.action = na.omit)
  phi <- coef(phiReg)
  gc <- gc - phi[1] - phi[2] * c - phi[3] * c^2
  kt[2, ] <- kt[2, ] + 2 * phi[3] * t
  kt[1, ] <- kt[1, ] + phi[2] * t + phi[3] * (t^2 - 2 * xbar * t)
  ax <- ax + phi[1] - phi[2] * x + phi[3] * x^2
  #\sum kt[i, ] = 0
  ci <- rowMeans(kt, na.rm = TRUE)
  ax <- ax + ci[1] + ci[2] * (xbar - x) + ci[3] * pmax(xbar - x, 0)
  kt[1, ] <- kt[1, ] - ci[1]
  kt[2, ] <- kt[2, ] - ci[2]
  kt[3, ] <- kt[3, ] - ci[3]
  list(ax = ax, bx = bx, kt = kt, b0x = b0x, gc = gc)

In [ ]:
PLAT <- StMoMo(link = "log", staticAgeFun = TRUE, periodAgeFun = c("1", f2, f3), 
               cohortAgeFun = "1", constFun = constPlat)

### Fitting Model

In [ ]:
PLATM <- fit(PLAT, data = EWMaleCenData, ages.fit = ages, years.fit = years, wxt = wxt)
PLATF <- fit(PLAT, data = EWFemaleCenData, ages.fit = ages, years.fit = years, wxt = wxt)

### Quality of Fit

In [ ]:
BIC(PLATM)
BIC(PLATF)

PLATM$deviance
PLATF$deviance

### Fitting Arima Models for kt1, kt2, kt3

In [ ]:
PLATM_list = list(ax = PLATM$ax, kt = PLATM$kt, gc = PLATM$gc)
PLATF_list = list(ax = PLATF$ax, kt = PLATF$kt, gc = PLATF$gc)

In [ ]:
(kt1M <- arima(diff(PLATM$kt[1, ]), order = c(0, 0, 0), include.mean = TRUE, method = "ML"))
(kt1F <- arima(diff(PLATF$kt[1, ]), order = c(0, 0, 0), include.mean = TRUE, method = "ML"))

(kt2M <- arima(PLATM$kt[2, ], order = c(1, 0, 0), include.mean = FALSE, method = "ML"))
(kt2F <- arima(PLATF$kt[2, ], order = c(1, 0, 0), include.mean = FALSE, method = "ML"))

(kt3M <- arima(PLATM$kt[3, ], order = c(1, 0, 0), include.mean = FALSE, method = "ML"))
(kt3F <- arima(PLATF$kt[3, ], order = c(1, 0, 0), include.mean = FALSE, method = "ML"))

(gcM <- arima(PLATM$gc, order = c(1, 0, 0), include.mean=FALSE, method = "ML"))
(gcF <- arima(PLATF$gc, order = c(1, 0, 0), include.mean=FALSE, method = "ML"))

## Save Model

In [ ]:

collect_arima_info <- function(y, model, manual_diff_order=0){
  order <- c(model$call$order[[2]], model$call$order[[3]], model$call$order[[4]])
  order[2] <- order[2] + manual_diff_order
  collected <- list(order=order, y=y, e=append(rep(0, manual_diff_order), as.vector(model$residuals)))
  if (order[1] >= 1){
    collected[["phi"]] <- as.vector(model$coef[1:order[1]])
  } else{
    collected[["phi"]] <- 0
  }
  
  if (order[3] >= 1){
    collected[["theta"]] <- as.vector(model$coef[(order[1]+1):(order[1]+order[3])])
  } else{
    collected[["theta"]] <- 0
  }
  
  collected[["mu"]] <- ifelse(any("intercept" %in% names(model$coef)), model$coef[["intercept"]], 0)
  collected[["sigma"]] <- model$sigma
  
  # intercept and mean are not the same (https://www.stat.pitt.edu/stoffer/tsa2/Rissues.htm)
  # only for ar1 
  # if (collected[["mu"]]!=0 & order[1]==1){
  #   collected[["mu"]] <- collected[["mu"]]*(1-collected[["phi"]][1])
  # }
  
  return(collected)

In [ ]:
PLATM_list[["kt"]] <- list(collect_arima_info(y=PLATM_list[["kt"]][1, ], model=kt1M, manual_diff_order = 1),
                      collect_arima_info(y=PLATM_list[["kt"]][2, ], model=kt2M),
                      collect_arima_info(y=PLATM_list[["kt"]][3, ], model=kt3M))

PLATM_list[["gc"]] <- collect_arima_info(y=PLATM_list[["gc"]], model=gcM)

PLATF[["kt"]] <- list(collect_arima_info(y=PLATF[["kt"]][1, ], model=kt1F, manual_diff_order = 1),
                      collect_arima_info(y=PLATF[["kt"]][2, ], model=kt2F),
                      collect_arima_info(y=PLATF[["kt"]][3, ], model=kt3F))

PLATF[["gc"]] <- collect_arima_info(y=PLATF[["gc"]], model=gcM)

In [ ]:
save(PLATM, PLATF, file = "PLAT.RData", compress = "gzip")